In [1]:
import os  
import sys  
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf  
from keras.backend.tensorflow_backend import set_session  
from keras.utils import multi_gpu_model
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
set_session(tf.Session(config=config))  
RAW_FILE_PATH = '../data/'

/home/user/cuser/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/user/cuser/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import keras.backend as K
from keras import layers
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Reshape, Add, Dropout
from keras.layers import Flatten, merge, Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model
# from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from keras.callbacks import EarlyStopping
import random
from keras.callbacks import History
from keras.utils import multi_gpu_model
import keras
import pickle

In [3]:
import seaborn as sns
# from sklearn.model_selection import train_test_split
import gc
import os
from sklearn.metrics import roc_auc_score
from collections import Counter
import matplotlib.pyplot as plt

%matplotlib inline
#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
import sys
sys.path.append('/home1/cuser/zero/tecent/')

In [5]:
fnl_seed = 2018
np.random.seed(fnl_seed)
os.environ['PYTHONHASHSEED'] = '2018'
random.seed(fnl_seed)
tf.set_random_seed(fnl_seed)

## Data preprocessing

In [12]:
train = pd.read_csv('train_kflod.csv')
test = pd.read_csv('test_kflod_new_b.csv')

train.shape
test.shape

(8798814, 156)

(2265989, 156)

In [13]:
df = pd.concat([train, test])

df.shape

del train
del test
gc.collect()

(11064803, 156)

7

In [14]:
df.reset_index(drop=True, inplace=True)
df.uid = df.uid.astype('int64')

# df = df.merge(user, on=['uid'], how='left')
df.shape

del user
gc.collect()

(11064803, 156)

15

In [15]:
df.to_csv('df_b.csv', index=None)

In [21]:
df.groupby(['label']).aid.count()

label
-99.0    2265989
-1.0     8376853
 1.0      421961
Name: aid, dtype: int64

## normalization

In [6]:
df = pd.read_csv('df_b.csv')
df['label'][df.label == -1] = 0

df.shape

(11064803, 156)

In [7]:
ad_feature = pd.read_csv('../adFeature.csv')

ad_feature.shape
ad_feature.head()

(173, 8)

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,8203,76104,1500666,59,282,0,6
1,2050,19441,178687,245165,53,1,0,6
2,1716,5552,158101,1080850,35,27,113,9
3,336,370,4833,119845,22,67,113,9
4,671,45705,352827,660519,42,67,0,4


In [8]:
df = df.merge(ad_feature, on=['aid'], how='left')

df.shape

(11064803, 163)

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=5, random_state=2018)

# train_index, val_index = [], []
# for crt_aid in ad_feature['aid'].values:
#     crt_train = train[train['aid']==crt_aid].copy()
#     crt_train.reset_index(drop=True, inplace=True)
#     for crt_train_index, crt_val_index in skf.split(crt_train, crt_train['label']):
#         train_index += crt_train.iloc[crt_train_index]['data_index'].values.tolist()
#         val_index += crt_train.iloc[crt_val_index]['data_index'].values.tolist()
#         break
# print(len(train_index), len(val_index), len(set(train_index) & set(val_index)), 
#       len(set(range(train.shape[0])) - set(train_index) - set(val_index)))

# del ad_feature
# del train

In [9]:
from models.official import wide_deep

In [6]:
# 用官方的评分方法计算AUC
def cal_tecent_auc(data, val_pred):
    assert(type(data) == pd.core.frame.DataFrame and 'aid' in data.columns and 'uid' in data.columns)
    
    df_eval = data[['aid', 'label']].copy()
    assert(len(val_pred) == df_eval.shape[0])
    df_eval['pred'] = val_pred
    return np.mean(df_eval.groupby('aid').apply(lambda x:roc_auc_score(x['label'], x['pred'])))

In [7]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape = (1, ), dtype = 'int64', name = name)
#     if n_out>8: n_out//=2
#     n_out = 5
    return inp, Embedding(n_in, n_out, input_length = 1, embeddings_initializer=keras.initializers.uniform(seed=fnl_seed),
                         trainable=True)(inp)

def continous_input(name):
    inp = Input(shape=(1, ), dtype = 'float32', name = name)
    return inp, Reshape((1, 1))(inp)



# similar to https://github.com/jrzaurin/Wide-and-Deep-Keras/blob/master/wide_and_deep_keras.py
def feature_generate(data, cate_columns, cont_columns, wide_columns=[]):
    embeddings_tensors = []
    continuous_tensors = []
    
    for ec in cate_columns:
        layer_name = ec + '_inp'
        layer_name = layer_name.replace('+', '_')
        layer_name = layer_name.replace('%', '_')
        # For categorical features, we em-bed the features in dense vectors of dimension 6×(category cardinality)**(1/4)
        embed_dim = data[ec].nunique() if int(6 * np.power(data[ec].nunique(), 1/4)) > data[ec].nunique() else int(6 * np.power(data[ec].nunique(), 1/4))
#         print(ec, embed_dim)
        t_inp, t_build = embedding_input(layer_name, data[ec].nunique(), embed_dim)
        embeddings_tensors.append((t_inp, t_build))
        del(t_inp, t_build)
        
    for cc in cont_columns:
        layer_name = cc + '_inp'
        layer_name = layer_name.replace('+', '_')
        layer_name = layer_name.replace('%', '_')
        t_inp, t_build = continous_input(layer_name)
        continuous_tensors.append((t_inp, t_build))
        del(t_inp, t_build)
        
    for wc in wide_columns:
        layer_name = 'w_' + wc + '_inp'
        layer_name = layer_name.replace('+', '_')
        layer_name = layer_name.replace('%', '_')
        t_inp, t_build = continous_input(layer_name)
        continuous_tensors.append((t_inp, t_build))
        del(t_inp, t_build)
    
    inp_layer =  [et[0] for et in embeddings_tensors]
    inp_layer += [ct[0] for ct in continuous_tensors]
    inp_embed =  [et[1] for et in embeddings_tensors]
    inp_embed += [ct[1] for ct in continuous_tensors]
    
    return inp_layer, inp_embed

In [8]:
def fit(inp_layer, inp_embed, wide_feas_cnt=-1 ,deep_layer_sizes=[256, 128, 64, 32], cross_layer_cnt=1, adam_r=0.05,
        drop_out_r = None, use_bn=False, grad_cliping_norm=None, is_print_sum=False, use_multi_gpu = False):
    
    if cross_layer_cnt:
        assert((type(cross_layer_cnt) == int) and cross_layer_cnt > 0)
        
    print('deep_layer_sizes:', deep_layer_sizes, ', cross_layer_cnt:',cross_layer_cnt, ', adam_r:', adam_r, 
          ', drop_out_r:',drop_out_r, ', use_bn:', use_bn, ', grad_cliping_norm:', grad_cliping_norm)
    

#     input = merge(inp_embed, mode = 'concat')
    if wide_feas_cnt > 0:
        input = merge(inp_embed[:-wide_feas_cnt], mode = 'concat')
    else:
        input = merge(inp_embed, mode = 'concat')
        
    
#     input = BatchNormalization()(input) #不宜在这里做BN，因为embedding之后的数值相差很大
    print("int_shape(input) :", K.int_shape(input))
    # deep layer
    for i, node_cnt in enumerate(deep_layer_sizes):
        if i == 0:
            deep = Dense(node_cnt, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                        bias_initializer=keras.initializers.Constant(value=0.1))(Flatten()(input))
            if use_bn: 
                deep = BatchNormalization()(deep)
            if drop_out_r != None:
                assert(type(drop_out_r) == float)
                deep = Dropout(drop_out_r, seed=fnl_seed)(deep)
        else:
            deep = Dense(node_cnt, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                        bias_initializer=keras.initializers.Constant(value=0.1))(deep)
            if use_bn:
                deep = BatchNormalization()(deep)
            if drop_out_r != None:
                assert(type(drop_out_r) == float)
                deep = Dropout(drop_out_r, seed=fnl_seed)(deep)

    # cross layer
    if cross_layer_cnt:
        cross = CrossLayer(output_dim = input.shape[2].value, num_layer = cross_layer_cnt, name = "cross_layer")(input)

    #concat both layers
    
    if cross_layer_cnt:
        if wide_feas_cnt > 0:
            wide_input = merge(inp_embed[-wide_feas_cnt:], mode = 'concat')
            output = merge([deep, cross, Flatten()(wide_input)], mode = 'concat')
        else:
            output = merge([deep, cross], mode = 'concat')
            
    else:
        output = deep
    output = Dense(1, activation = 'sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                        bias_initializer=keras.initializers.Constant(value=0.1))(output)
    model = Model(inp_layer, output)
    if is_print_sum: 
        print(model.summary())
#         plot_model(model, to_file = 'model.png', show_shapes = True)
    
    if use_multi_gpu:
        try:
            model = multi_gpu_model(model, gpus=4)
            print("Training using multiple GPUs..")
        except Exception as e:
            print("Training using single GPU or CPU..")
            print(e)
    
    if grad_cliping_norm:
        assert(grad_cliping_norm > 0)
        model.compile(Adam(adam_r, clipnorm=grad_cliping_norm), loss = 'binary_crossentropy', metrics = [])

    else:
        model.compile(Adam(adam_r), loss = 'binary_crossentropy', metrics = [])
    print('model total params:', model.count_params())
    
    return model


class CrossLayer(layers.Layer):
    def __init__(self, output_dim, num_layer, **kwargs):
        self.output_dim = output_dim
        self.num_layer = num_layer
        super(CrossLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dim = input_shape[2]
        self.W = []
        self.bias = []
        for i in range(self.num_layer):
            self.W.append(self.add_weight(shape = [1, self.input_dim], initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                                          name = 'w_' + str(i), trainable = True))
            self.bias.append(self.add_weight(shape = [1, self.input_dim], initializer = 'zeros', name = 'b_' + str(i), trainable = True))
        self.built = True

    def call(self, inputs):
        for i in range(self.num_layer):
            if i == 0:
#                 cross = Lambda(lambda x: Add()([K.sum(self.W[i] * K.batch_dot(K.reshape(x, (-1, self.input_dim, 1)), x), 1, keepdims = True), self.bias[i], x]))(input)
                cross = Lambda(lambda x: Add()([tf.transpose(K.sum(self.W[i] * K.batch_dot(K.reshape(x, (-1, self.input_dim, 1)), x), 2, keepdims = True), perm=[0,2,1]), self.bias[i], x]))(inputs)  
            else:
                cross = Lambda(lambda x: Add()([tf.transpose(K.sum(self.W[i] * K.batch_dot(K.reshape(x, (-1, self.input_dim, 1)), inputs), 2, keepdims = True), perm=[0,2,1]), self.bias[i], inputs]))(cross)
        return Flatten()(cross)

    def compute_output_shape(self, input_shape):
        return (None, self.output_dim) 

In [10]:
def build_input_data(data, concat_files, suffix, verbose=False, exclude_cols=[], nrows=None, skip_nrows=None):
    if verbose: print('data.shape:', data.shape)
    for file_path in concat_files:
        file_path = './nn_input'+file_path[file_path.rfind('/'):file_path.rfind('.')]+suffix
        data = concat_data(data, file_path, exclude_cols=exclude_cols, verbose=verbose, nrows=nrows,
                          skip_nrows=skip_nrows)
        if verbose: print('data.shape:', data.shape)
    return data

In [19]:
def train_model(model, train_val, is_adjust_params, adjust_params_val_len=500000, adjust_params_batch_cnt=2, 
                eopch_rows = 1000000, epoch_cnt=100, is_train_with_val=False, batch_size=1024, ):
    max_auc, crt_auc = 0.5, 0.5
    times =  train_val.shape[0] // eopch_rows + 1
    times = min(adjust_params_batch_cnt, times) if is_adjust_params else times
#     train_val['aid'] = train_val['aid'].map(aid_dict)
    
#     is_adjust_params #验证集只取100000，只用一个batch训练，用 nomal_auc 决定是否继续下一轮训练
    if not is_train_with_val:
        if is_adjust_params:
            validation_data=([train_val[:adjust_params_val_len][c].values for c in use_feas], 
                             train_val[:adjust_params_val_len]['label'].astype(int).values.reshape(-1, 1))
        else:
            validation_data=([train_val[c].values for c in use_feas], 
                             train_val['label'].astype(int).values.reshape(-1, 1))


    for crt_epoch in range(1, epoch_cnt+1): # epoch
        if is_train_with_val:
            print('training with val')
            model.fit([train_val[c].values for c in use_feas], train_val['label'].astype(int).values.reshape(-1, 1),
                  batch_size=512, epochs=1, shuffle=True)
        for crt_time in range(times): #batch
            print('epoch:', crt_epoch, ', batch:', crt_time+1)
            batch_start, batch_end = crt_time*eopch_rows, (crt_time+1)*eopch_rows
            df_batch = train_train[batch_start:batch_end].copy()
            df_batch.reset_index(drop=True, inplace=True)

            model.fit([df_batch[c].values for c in use_feas], df_batch['label'].astype(int).values.reshape(-1, 1),
                  batch_size=batch_size, epochs=1, shuffle=True)
        
                
        if not is_train_with_val:
            pred = model.predict(validation_data[0])
            normal_crt_auc = roc_auc_score(validation_data[1], pred)

            if is_adjust_params: 
                crt_auc = cal_tecent_auc(train_val[:adjust_params_val_len][['aid', 'uid', 'label']], pred)
            else:
                crt_auc = cal_tecent_auc(train_val[['aid', 'uid', 'label']], pred)

            crt_log_loss = log_loss(validation_data[1], pred)
            print('x'*10, 'epoch', crt_epoch,',t_auc =', crt_auc, ',normal_auc =', normal_crt_auc, ', log_loss =', crt_log_loss, 'x'*10)
            if crt_auc<=max_auc:
                print('early stopping,best epoch =', crt_epoch-1)
                break
            else:
                max_auc = crt_auc
                
    if not is_train_with_val:
        return model, crt_epoch-1, max_auc 
    else:
        pred = model.predict([train_val[c].values for c in use_feas])
        normal_crt_auc = roc_auc_score(train_val['label'].astype(int).values.reshape(-1, 1), pred)
        crt_auc = cal_tecent_auc(train_val[['aid', 'uid', 'label']], pred)
        print('refit, t_auc={}, normal_auc'.format(crt_auc, normal_crt_auc))
        return model, crt_epoch, crt_auc

In [13]:
def gen_transform_tool(data, save_route = './transform_tool/', cate_feas=[], conti_feas=[]):
    for fea in data.columns:
        if fea in cate_feas:
            uniq = data[fea].unique()
            mapping = dict(zip(uniq,range(1,len(uniq) + 1)))
            print('dict:', fea, '->', save_route+fea+'.dict')
            pickle.dump(mapping, open(save_route+fea+'.dict', 'wb'))
            data[fea] = data[fea].map(mapping)
        elif fea in conti_feas:
            scaler_val = data[fea][~data[fea].isnull()].values
            scaler = StandardScaler().fit(scaler_val.reshape((len(scaler_val), 1)))
            print('scaler:', fea, '->', save_route+fea+'.scaler')
            pickle.dump(scaler, open(save_route+fea+'.scaler', 'wb'))
            data[fea].fillna(scaler.mean_[0], inplace=True)
            data[fea] = scaler.transform(data[fea].values.reshape((len(data), 1))).reshape((len(data),)).tolist()
    return data

In [17]:
df.head()

,aid,uid,label,age,gender,education,consumptionAbility,LBS,carrier,house,...,house_pos_std,house_mean_sub,house_mean_pos_sub,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,69569679,0.0,4.0,1.0,6.0,0.0,177.0,1.0,0.0,...,0.451669,0.275193,0.284658,8203,76104,1500666,59,282,0,6
1,177,66984139,0.0,5.0,1.0,7.0,1.0,94.0,3.0,1.0,...,0.451669,0.724807,0.715342,8203,76104,1500666,59,282,0,6
2,177,39132943,0.0,2.0,1.0,4.0,1.0,921.0,2.0,1.0,...,0.451669,0.724807,0.715342,8203,76104,1500666,59,282,0,6
3,177,34456255,0.0,2.0,1.0,2.0,2.0,464.0,1.0,1.0,...,0.451669,0.724807,0.715342,8203,76104,1500666,59,282,0,6
4,177,3036262,1.0,4.0,2.0,6.0,0.0,585.0,1.0,0.0,...,0.451669,0.275193,0.284658,8203,76104,1500666,59,282,0,6


In [18]:
gen_transform_tool(df, cate_feas=cate_feas, conti_feas=conti_feas)

dict: age -> ./transform_tool/age.dict
dict: gender -> ./transform_tool/gender.dict
dict: education -> ./transform_tool/education.dict
dict: consumptionAbility -> ./transform_tool/consumptionAbility.dict
dict: carrier -> ./transform_tool/carrier.dict
dict: house -> ./transform_tool/house.dict
scaler: marriageStatus_split_count_score -> ./transform_tool/marriageStatus_split_count_score.scaler
scaler: marriageStatus_split_pos_count_score -> ./transform_tool/marriageStatus_split_pos_count_score.scaler
scaler: interest1_split_count_score -> ./transform_tool/interest1_split_count_score.scaler
scaler: interest1_split_pos_count_score -> ./transform_tool/interest1_split_pos_count_score.scaler
scaler: interest2_split_count_score -> ./transform_tool/interest2_split_count_score.scaler
scaler: interest2_split_pos_count_score -> ./transform_tool/interest2_split_pos_count_score.scaler
scaler: interest3_split_count_score -> ./transform_tool/interest3_split_count_score.scaler
scaler: interest3_split_p

scaler: gender_pos_max -> ./transform_tool/gender_pos_max.scaler
scaler: gender_pos_min -> ./transform_tool/gender_pos_min.scaler
scaler: gender_pos_mean -> ./transform_tool/gender_pos_mean.scaler
scaler: gender_pos_std -> ./transform_tool/gender_pos_std.scaler
scaler: gender_mean_sub -> ./transform_tool/gender_mean_sub.scaler
scaler: gender_mean_pos_sub -> ./transform_tool/gender_mean_pos_sub.scaler
scaler: education_max -> ./transform_tool/education_max.scaler
scaler: education_min -> ./transform_tool/education_min.scaler
scaler: education_mean -> ./transform_tool/education_mean.scaler
scaler: education_std -> ./transform_tool/education_std.scaler
scaler: education_pos_max -> ./transform_tool/education_pos_max.scaler
scaler: education_pos_min -> ./transform_tool/education_pos_min.scaler
scaler: education_pos_mean -> ./transform_tool/education_pos_mean.scaler
scaler: education_pos_std -> ./transform_tool/education_pos_std.scaler
scaler: education_mean_sub -> ./transform_tool/education

,aid,uid,label,age,gender,education,consumptionAbility,LBS,carrier,house,...,house_pos_std,house_mean_sub,house_mean_pos_sub,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,69569679,0.0,1,1,1,1,177.0,1,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1
1,177,66984139,0.0,2,1,2,2,94.0,2,2,...,1.221449,1.785229,1.687917,1,1,1,1,1,1,1
2,177,39132943,0.0,3,1,3,2,921.0,3,2,...,1.221449,1.785229,1.687917,1,1,1,1,1,1,1
3,177,34456255,0.0,3,1,4,3,464.0,1,2,...,1.221449,1.785229,1.687917,1,1,1,1,1,1,1
4,177,3036262,1.0,1,2,1,1,585.0,1,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1
5,177,81723792,0.0,2,2,2,2,585.0,1,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1
6,177,38276996,0.0,2,2,1,2,809.0,1,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1
7,177,27285489,0.0,1,2,5,2,112.0,1,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1
8,177,24170811,0.0,1,2,3,2,346.0,4,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1
9,177,58509493,0.0,4,2,5,2,486.0,3,1,...,1.221449,-0.027857,0.054644,1,1,1,1,1,1,1


In [19]:
df.to_csv('df_b_trans.csv', index=None)

## train

In [11]:
df = pd.read_csv('df_b.csv', nrows=5)

df.head()

,aid,uid,label,age,gender,education,consumptionAbility,LBS,carrier,house,marriageStatus_split_count_score,marriageStatus_split_pos_count_score,interest1_split_count_score,interest1_split_pos_count_score,interest2_split_count_score,interest2_split_pos_count_score,interest3_split_count_score,interest3_split_pos_count_score,interest4_split_count_score,interest4_split_pos_count_score,interest5_split_count_score,interest5_split_pos_count_score,kw1_split_count_score,kw1_split_pos_count_score,kw2_split_count_score,kw2_split_pos_count_score,kw3_split_count_score,kw3_split_pos_count_score,topic1_split_count_score,topic1_split_pos_count_score,topic2_split_count_score,topic2_split_pos_count_score,topic3_split_count_score,topic3_split_pos_count_score,appIdInstall_split_count_score,appIdInstall_split_pos_count_score,appIdAction_split_count_score,appIdAction_split_pos_count_score,ct_split_count_score,ct_split_pos_count_score,os_split_count_score,os_split_pos_count_score,age_feature_count_score,age_feature_pos_count_score,gender_feature_count_score,gender_feature_pos_count_score,marriageStatus_feature_count_score,marriageStatus_feature_pos_count_score,education_feature_count_score,education_feature_pos_count_score,consumptionAbility_feature_count_score,consumptionAbility_feature_pos_count_score,LBS_feature_count_score,LBS_feature_pos_count_score,interest1_feature_count_score,interest1_feature_pos_count_score,interest2_feature_count_score,interest2_feature_pos_count_score,interest3_feature_count_score,interest3_feature_pos_count_score,interest4_feature_count_score,interest4_feature_pos_count_score,interest5_feature_count_score,interest5_feature_pos_count_score,kw1_feature_count_score,kw1_feature_pos_count_score,kw2_feature_count_score,kw2_feature_pos_count_score,kw3_feature_count_score,kw3_feature_pos_count_score,topic1_feature_count_score,topic1_feature_pos_count_score,topic2_feature_count_score,topic2_feature_pos_count_score,topic3_feature_count_score,topic3_feature_pos_count_score,appIdInstall_feature_count_score,appIdInstall_feature_pos_count_score,appIdAction_feature_count_score,appIdAction_feature_pos_count_score,ct_feature_count_score,ct_feature_pos_count_score,os_feature_count_score,os_feature_pos_count_score,carrier_feature_count_score,carrier_feature_pos_count_score,age_max,age_min,age_mean,age_std,age_pos_max,age_pos_min,age_pos_mean,age_pos_std,age_mean_sub,age_mean_pos_sub,gender_max,gender_min,gender_mean,gender_std,gender_pos_max,gender_pos_min,gender_pos_mean,gender_pos_std,gender_mean_sub,gender_mean_pos_sub,education_max,education_min,education_mean,education_std,education_pos_max,education_pos_min,education_pos_mean,education_pos_std,education_mean_sub,education_mean_pos_sub,consumptionAbility_max,consumptionAbility_min,consumptionAbility_mean,consumptionAbility_std,consumptionAbility_pos_max,consumptionAbility_pos_min,consumptionAbility_pos_mean,consumptionAbility_pos_std,consumptionAbility_mean_sub,consumptionAbility_mean_pos_sub,LBS_max,LBS_min,LBS_mean,LBS_std,LBS_pos_max,LBS_pos_min,LBS_pos_mean,LBS_pos_std,LBS_mean_sub,LBS_mean_pos_sub,carrier_max,carrier_min,carrier_mean,carrier_std,carrier_pos_max,carrier_pos_min,carrier_pos_mean,carrier_pos_std,carrier_mean_sub,carrier_mean_pos_sub,house_max,house_min,house_mean,house_std,house_pos_max,house_pos_min,house_pos_mean,house_pos_std,house_mean_sub,house_mean_pos_sub
0,177.0,69569679,-1.0,4.0,1.0,6.0,0.0,177.0,1.0,0.0,2105896.0,87419.0,55227162.0,2508966.0,4225919.0,269387.0,0.0,0.0,0.0,0.0,49632388.0,2242622.0,18351.0,739.0,916634.0,45845.0,0.0,0.0,2164218.0,102632.0,10672.0,490.0,0.0,0.0,0.0,0.0,0.0,0.0,10958079.0,499996.0,4690694.0,231317.0,1349047.0,52831.0,4587739.0,227293.0,2105896.0,87419.0,1214906.0,55527.0,1506621.0,63461.0,17070.0,802.0,0.0,0.0,592.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1607163.0,71174.0,4163659.0,209714.0,3394348.0,155795.0,5.0,1.0,2.970611,1.660646,5.0,1.0,3.022181,1.6875,1.029389,0.977

In [15]:
train_train = df[df.label != -99].copy()

train_train.shape
gc.collect()

(8798814, 163)

7

In [16]:
by = ['aid', 'uid']
target = ['label']
origin = ['age', 'gender', 'education', 'consumptionAbility', 'LBS', 'carrier', 'house', 'advertiserId', 'campaignId', 'creativeId', 'creativeSize', 'adCategoryId', 'productId', 'productType']
predictors = list(set(df.columns) - set(by) - set(target) - set(origin))

In [17]:
cate_feas = ['age', 'gender', 'education', 'consumptionAbility', 'carrier', 'house', 'advertiserId', 'campaignId', 'creativeId', 'creativeSize', 'adCategoryId', 'productId', 'productType']
conti_feas = predictors
wide_feas = []
use_feas = cate_feas + conti_feas + wide_feas

print('(use_feas, cate_feas, conti_feas) len :', len(use_feas), len(cate_feas), len(conti_feas), len(wide_feas))

(use_feas, cate_feas, conti_feas) len : 159 13 146 0


In [20]:
print('training submition model')
K.clear_session()
gc.collect()
inp_layer, inp_embed = feature_generate(df, cate_feas, conti_feas, wide_feas)
model = fit(inp_layer, inp_embed, wide_feas_cnt=len(wide_feas),
            deep_layer_sizes=[2**10, 2**10], cross_layer_cnt=0, adam_r=0.01,
            drop_out_r = 0.5, use_bn=True , is_print_sum=False, grad_cliping_norm=None)
model, best_epoch, best_score = train_model(model, is_adjust_params=False, train_val=df[df.label != -99].iloc[-1000000:, :].copy(), epoch_cnt=7, 
                                            is_train_with_val=True, batch_size=1024)
print('best_epoch={}, best_score={}'.format(best_epoch, best_score))
model.save('./model/dcn_4best.model')

training submition model


21144

deep_layer_sizes: [1024, 1024] , cross_layer_cnt: 0 , adam_r: 0.01 , drop_out_r: 0.5 , use_bn: True , grad_cliping_norm: None
int_shape(input) : (None, 1, 274)
model total params: 1349534
training with val
Epoch 1/1
1000000/1000000 [==============================] - 103s 103us/step - loss: 0.2067
epoch: 1 , batch: 1
Epoch 1/1
1000000/1000000 [==============================] - 53s 53us/step - loss: 0.1934
epoch: 1 , batch: 2
Epoch 1/1
1000000/1000000 [==============================] - 63s 63us/step - loss: 0.1903
training with val
Epoch 1/1
1000000/1000000 [==============================] - 96s 96us/step - loss: 0.1920
epoch: 2 , batch: 1
Epoch 1/1
1000000/1000000 [==============================] - 53s 53us/step - loss: 0.1866
epoch: 2 , batch: 2
Epoch 1/1
1000000/1000000 [==============================] - 53s 53us/step - loss: 0.1861
training with val
Epoch 1/1
1000000/1000000 [==============================] - 98s 98us/step - loss: 0.1898
epoch: 3 , batch: 1
Epoch 1/1
1000000/1000000 

OSError: Unable to create file (unable to open file: name = './model/dcn_4best.model', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [23]:
model.save('dcn.model')

## predict

In [22]:
test=df[df.label == -99]
test.shape

(2265989, 163)

In [24]:
pred = model.predict([test[c].values for c in use_feas])
print(len(pred))

2265989


In [25]:
print(len(use_feas))
submit = test[['aid','uid']].copy()
submit['score'] = pred
submit['score'] = submit['score'].apply(lambda x: float('%.8f' % x))
print(submit.shape)
submit.head()

159
(2265989, 3)


,aid,uid,score
8798814,2118.0,64355836,0.079282
8798815,692.0,45051997,0.053348
8798816,692.0,10869198,0.044588
8798817,1918.0,75929554,0.039235
8798818,1596.0,5790162,0.180931


In [26]:
sub_zip_name = 'dcn_best'
submit.to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')
del sub_zip_name

0

0

0